In [149]:
from PIL import Image
from PIL import Image, ImageDraw
import random as r
import os
import numpy as np
import ghalton
from tqdm import tqdm
from scipy.optimize import fmin, minimize
from math import fabs
import time

In [150]:
def scale_rotate(image,coeff,rotate,force=False):#img,coeff - how to scale 1 norm, rotate - angle of rotation
    if force != True:#если нужно что-то руками отредачить 
        if coeff <= 0.8:#Да так кастыльно установил границы на скейл
            coeff = 0.8
        if coeff >=1.4:
            coeff = 1.4

        if rotate >= 20:
            rotate = 20
        if rotate <= -20:
            rotate = -20

#     im = image.rotate(rotate, expand=True)#
    im = image.rotate(rotate, expand=True, resample = 2)#Экспериментальное говно
    im = im.resize((int(image.size[0]*fabs(coeff)),int(image.size[1]*fabs(coeff))))
    
    return im


In [151]:
def rescale(val_x, val_y, in_min, in_max, out_min, out_max_x, out_max_y):
    x = out_min + (val_x - in_min) * ((out_max_x - out_min) / (in_max - in_min))
    y = out_min + (val_y - in_min) * ((out_max_y - out_min) / (in_max - in_min))
    return round(x), round(y)

In [152]:
def random_point(dir_name = "crop"):
    main_image = Image.new('RGBA', (842, 1192))# Вредно для маштабирования!!!
    file = os.listdir(dir_name)
    sequencer = ghalton.Halton(2)#Генерируем вектор псевдослуч
    points = sequencer.get(len(file))
    
    points_random =[]
    randomisation = np.random.random() 
    for i in points:#Добовляем случайности в псевдослучайный вектор
        uniforms_x = (randomisation + i[0]) % 1 
        uniforms_y = (randomisation + i[1]) % 1
        points_random.append((uniforms_x,uniforms_y))
    #Normalization Data Or Rescale To Image Size
    points_clear=[]   
    # Добавляю ограничение чтобы фото не вылезали за границу плаката 
    x = main_image.size[1] -400# изменяя эти параметры можно регулировать границы раднома, положение верт метки
    y = main_image.size[0]//2 # высота горизонтальной метки 
    for i in points_random:
#                                         нормализация        рандомное scale      Рандомный rotate
        points_clear.append((*rescale(*i, 0, 1, 0, x, y), r.uniform(1, 1.4),np.random.normal(0,0.5)))
    return points_clear

In [160]:
points =  np.array(random_point())
points.shape

(14, 4)

In [189]:
dir_name = "crop"
file = os.listdir(dir_name)
count_file = len(file)
out_filename = 0
iter_to_photo = 0


#make_photo нужна что бы не плодить дубликаты при множественном вызове 
picture_box = []
for i in file:
    picture_box.append(Image.open(os.path.abspath(dir_name)+"/"+i))

# Нужно выпилить постоянную загрузку изображений в память 
def main_for_optimization(point,make_photo=True): # нужно передавать массив с аргументами
    global out_filename
    main_image= Image.new('RGBA', (842, 1192)) # понизил разрешение в 2 раза для скорости
    count = 0# для перебора координат в points_clear
    #Без нее не работает оптимизация потому,что ветор который выдает оптимизатор видимо имеет одномерную размерность 
    point = point.reshape(count_file,4)
    for one_img in picture_box:
        now_img = scale_rotate(one_img,point[count][2],point[count][3]*10)#Посленяя это угол *10000 для скорости оптимизации
        main_image.paste(now_img, (int(point[count][1]),int(point[count][0])), now_img)#Numbers is coordinate x y 
        count+=1
#     main_image.show()
#Кидаем все варики орасположения с сильным сжатием 
    global iter_to_photo
    iter_to_photo +=1
    if make_photo and iter_to_photo%101 == 100:#С какой частотой делать фото 
        background = Image.new("RGB", main_image.size, (255, 255, 255))
        background.paste(main_image, mask=main_image.split()[3])
        background=scale_rotate(background,0.3,0,force=True)
        background.save("Powell_random/"+str(out_filename)+".jpg", 'JPEG', quality=20)
#         print("100 итераций пройдено, сохраняю фото")
        array = np.array(main_image)
        print((array.size - np.count_nonzero(array))/array.size)
        print("Num photo - "+str(out_filename))
        
#         print(point)
    out_filename +=1
    # На этом этапе изображение готово, далее оценщик
    array = np.array(main_image)
    return (array.size - np.count_nonzero(array))/array.size #Возвразаем (размер - не нулевые)/размер

In [165]:
# --- 0.3915524482727051 seconds --- Время для 14 фото 
#Время одного прохода функции
start_time = time.time()
main_for_optimization(points,make_photo=True)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.3889274597167969 seconds ---


In [196]:
TARGET = 0.1
out_filename = 0
Nfeval = 1
last_vec = np.array([])
def callbackF(Xi):
    global Nfeval
    global last_vec
    loss_values_now = main_for_optimization(Xi,make_photo=False)
#     if Nfeval % 10 == 10:#каждые 10 итераций показать что к чему
    print("past time is --- %s minute ---" % (time.time() - start_time))
    print ('{0:4d}   {1: 3.6f}'.format(Nfeval, loss_values_now))
    last_vec = Xi
    Nfeval += 1
    return True
    if loss_values_now<=TARGET:
        return True # если вернем true оптимизация закончится
print ('{0:4s}   {1:9s} '.format('Iter','f(X)') )

# fmin(main_for_optimization, points, callback=callbackF)
start_time = time.time()
# minimize(main_for_optimization, points,  method='Powell', callback=callbackF, options={'disp': True,'ftol': 0.005})
minimize(main_for_optimization, points,  method='Powell', callback=callbackF, options={'disp': True})
minimize

Iter   f(X)      
0.40489347032473016
0.3942522099029157
0.31894837316073904
0.3181261358382885
0.2777914222289531
0.2106666175134308
0.2106280089751152
0.11064858358972723
past time is --- 278.2160429954529 minute ---
   1    0.086144
0.0858337551212358


KeyboardInterrupt: 

In [197]:
#Оптимальное значение нужно поработать с настройкой отсечки 

callbackF(points)

past time is --- 873.746256351471 minute ---
   2    0.413346


True

In [108]:
main_for_optimization(res_fmin)

0.7397957633231839

In [36]:
main_for_optimization(x)

0.780968157670296

In [5]:
def download(username:str, count:int):
	res = os.system("instagram-scraper.exe -d photos/" + username + " --media-types image --latest --maximum " + str(count) + " " + username)
	if res:#if os.system return 1 
		print("Something wrong, check username!")
		raise ErrorUsername()

In [7]:
download("vinter_mute23123123123123",5)

In [183]:

x = Image.open('Powell_random/2173.jpg')
x =np.array(x)
(x.size - np.count_nonzero(x))/x.size


0.015761860299675426

In [185]:
0.01582-0.01576

6.0000000000001025e-05

In [ ]:
tol = 0.001